## Filtro gafas de sol con rotación

In [ ]:
import cv2
import numpy as np
import time
import FaceNormalizationUtils as faceutils
from FaceDetectors import FaceDetector
import math

# Initialize normalization and face detector
normalizatorHS = faceutils.Normalization()
FDet = FaceDetector()
font = cv2.FONT_HERSHEY_SIMPLEX

# Load glasses image with transparency (alpha channel)
glasses_img = cv2.imread("/home/m4rc/Desktop/VC/Ejercicios Practicas/Practica5/assets/filter_2_items/gafas.png", -1)

# Webcam connection
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print('Camera error')
        exit(0)

# Face and eye detection modes
imodoF = 0
imodoE = 0
debug = 0

# Set camera resolution
cap.set(3, 640)
cap.set(4, 480)

while True:
    t = time.time()
    ret, frame = cap.read()

    if ret:
        # Split channels for HS normalization
        B, G, R = cv2.split(frame)

        # Detect face and eyes
        values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
        if values is not None:
            face, eyes, shape = values
            [x, y, w, h] = face

            if x > -1:
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                [lex, ley, rex, rey] = eyes
                if lex > -1:
                    if imodoF > 0:
                        for (x, y) in shape:
                            cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)


                    # Calculate center between the eyes
                    center_x = (lex + rex) // 2
                    center_y = (ley + rey) // 2

                    # Apply scale factor to resize the glasses
                    scale_factor = 2.3  # Change this factor to adjust the glasses size
                    glasses_width = int((rex - lex) * scale_factor)
                    glasses_height = int(glasses_width / glasses_img.shape[1] * glasses_img.shape[0])

                    if glasses_width > 0 and glasses_height > 0:
                        # Resize glasses with the new dimensions
                        resized_glasses = cv2.resize(glasses_img, (glasses_width, glasses_height))

                        # Calculate the rotation angle based on the eye positions
                        delta_x = rex - lex
                        delta_y = rey - ley
                        angle = -math.degrees(math.atan2(delta_y, delta_x))

                        # Rotate the resized glasses around its center
                        center = (glasses_width // 2, glasses_height // 2)
                        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
                        rotated_glasses = cv2.warpAffine(resized_glasses, rotation_matrix, (glasses_width, glasses_height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)

                        # Calculate position to place glasses centered on the eyes
                        top_left_x = int(center_x - glasses_width / 2)
                        top_left_y = int(center_y - glasses_height / 2)

                        # Overlay the rotated glasses on the frame
                        for i in range(glasses_height):
                            for j in range(glasses_width):
                                if rotated_glasses[i, j, 3] > 0:  # Check the alpha channel for transparency
                                    y_offset = top_left_y + i
                                    x_offset = top_left_x + j
                                    if 0 <= y_offset < frame.shape[0] and 0 <= x_offset < frame.shape[1]:
                                        frame[y_offset, x_offset] = rotated_glasses[i, j, :3]

        if debug:
            print("Processing time : {:.3f}".format(time.time() - t))

        # Display face detector mode and frame
        cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        if imodoF == 1 or imodoF == 2:
            cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Cam', frame)

        # Control keys
        tec = cv2.waitKey(40)
        if tec & tec == 27:  # Esc to exit
            break

cap.release()
cv2.destroyAllWindows()
